# Practice Python JNB

Let's start by importing in the `R` function. We use `rpy2` to import the function and use it in `Python`.

In [3]:
import rpy2.robjects as ro
import pandas as pd
import json

r = ro.r
r['source']('ParseNFLPlayers.R')

parse_nfl_player_r = ro.globalenv['get_player_data']

# now we can use the function parse_nfl_player_r() to get info on players
# parse_nfl_player_r() returns an array of length one with the JSON data
# use parse_nfl_player_r("player_name")[0] to get a string of the JSON

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
* To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


R[write to console]: ── Attaching packages ──────────────────────────────────────── nflverse 1.0.1 ──

R[write to console]: ✔ nflfastR 4.3.0     ✔ nflreadr 1.1.3
✔ nflseedR 1.0.2     ✔ nflplotR 1.0.0
✔ nfl4th   1.0.1     

R[write to console]: ──────────────────────────────────────────────────────────────── Ready to go! ──

R[write to console]: Registered S3 method overwritten by 'tune':
  method                   from   
  required_pkgs.model_spec parsnip

R[write to console]: ── Attaching packages ────────────────────────────────────── tidymodels 0.1.4 ──

R[write to console]: ✔ broom        0.7.12     ✔ recipes      0.1.17
✔ dials        0.0.10     ✔ rsample      0.1.1 
✔ dplyr        1.0.7      ✔ tibble       3.1.6 
✔ ggplot2      3.3.5      ✔ tidyr        1.1.4 
✔ infer        1.0.0      ✔ tune         0.1.6 
✔ modeldata    0.1.1      ✔ workflows    0.2.4 
✔ parsnip      0.1.7      ✔ workflowsets 0.1.0 
✔ purrr        0.3.4      ✔ yardstick    0.0.9 

R[write to console]: ── Confl

Here is an example of manipulating some player data with `pandas`. Let's look at the data for Patrick Mahomes.

In [4]:
# Get the data from the R function
string_json_pmahomes = parse_nfl_player_r("Patrick Mahomes")[0]
# Parse the string into a JSON
json_pmahomes = json.loads(string_json_pmahomes)
print(json_pmahomes)

R[write to console]: New names:
* season -> season...1
* season -> season...26



[{'season...1': 2021, 'team': 'KC', 'position': 'QB', 'depth_chart_position': 'QB', 'jersey_number': 15, 'status': 'Active', 'full_name': 'Patrick Mahomes', 'first_name': 'Patrick', 'last_name': 'Mahomes', 'birth_date': '1995-09-17', 'height': '75', 'weight': '227', 'college': 'Texas Tech', 'high_school': 'Whitehouse (TX)', 'gsis_id': '00-0033873', 'espn_id': 3139477, 'sportradar_id': '11cad59d-90dd-449c-a839-dddaba4fe16c', 'yahoo_id': 30123, 'rotowire_id': 11839, 'pff_id': 11765, 'pfr_id': 'MahoPa00', 'fantasy_data_id': 18890, 'sleeper_id': '4046', 'years_exp': 5, 'headshot_url': 'https://static.www.nfl.com/image/private/f_auto,q_auto/league/tlg2oo5yrk6mins3nxxe', 'season...26': 2021, 'week': 10.95, 'completions': 26.25, 'attempts': 39, 'passing_yards': 294.8, 'passing_tds': 2.4, 'interceptions': 0.8, 'sacks': 1.85, 'sack_yards': 11.35, 'sack_fumbles': 0.35, 'sack_fumbles_lost': 0.1, 'passing_air_yards': 275.95, 'passing_yards_after_catch': 166.2, 'passing_first_downs': 15.8, 'passing

In [5]:
# Turn the JSON into a dataframe
df = pd.json_normalize(json_pmahomes)
# Display the dataframe
df.head()

,season...1,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,...,team_division,team_color,team_color2,team_color3,team_color4,team_logo_wikipedia,team_logo_espn,team_wordmark,team_conference_logo,team_league_logo
0,2021,KC,QB,QB,15,Active,Patrick Mahomes,Patrick,Mahomes,1995-09-17,...,AFC West,#E31837,#FFB612,#000000,#e31837,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/kc.png,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...


(The dataframe is pretty small - I hope to change the API soon to return more data rather than just averages.)

Instead, let's look at the data directly in `Python`. We get this from the same `R` file that was imported in the first code chunk.

In [6]:
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# This is an R "tibble" of the data
# We load this object from the R script we used earlier
player_stats_r = ro.globalenv['player_stats']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
  df_player_stats = ro.conversion.rpy2py(player_stats_r)

# View the first 5 rows of the data
df_player_stats.head()

,player_id,player_name,recent_team,season,week,season_type,completions,attempts,passing_yards,passing_tds,...,receiving_epa,receiving_2pt_conversions,special_teams_tds,fantasy_points,fantasy_points_ppr,pacr,racr,target_share,air_yards_share,wopr
1,00-0019596,T.Brady,TB,2021,1,REG,32,50,379.0,4,...,NaN,0,0.0,27.16,27.16,0.849776,NaN,NaN,NaN,NaN
2,00-0019596,T.Brady,TB,2021,2,REG,24,36,276.0,5,...,NaN,0,0.0,29.64,29.64,0.795389,NaN,NaN,NaN,NaN
3,00-0019596,T.Brady,TB,2021,3,REG,41,55,432.0,1,...,NaN,0,0.0,28.68,28.68,1.130890,NaN,NaN,NaN,NaN
4,00-0019596,T.Brady,TB,2021,4,REG,22,43,269.0,0,...,NaN,0,0.0,11.06,11.06,0.586057,NaN,NaN,NaN,NaN
5,00-0019596,T.Brady,TB,2021,5,REG,30,41,411.0,5,...,NaN,0,0.0,37.74,37.74,0.976247,NaN,NaN,NaN,NaN


Have fun :) If you want to check out other dataframes, `player_stats_kicking`, `team_data`, and `roster` are all tibbles in the `R` script. Import them into python using the same process that we used for `player_stats`.

Let me know if you want me to add some examples of how to manipulate data using Python!

In [25]:
df_player_stats[df_player_stats.recent_team == "TB"]
df_player_stats[["recent_team", "week"]]

,recent_team,week
1,TB,1
2,TB,2
3,TB,3
4,TB,4
5,TB,5
...,...,...
5676,NYJ,14
5677,NYJ,15
5678,NYJ,16
5679,NYJ,17
